In [1]:
from sqlalchemy import text
from app.database.database import engine_origen
from app.database.querys import query_bien_consumo
import pandas as pd

conn_origen = engine_origen.connect()
result = conn_origen.execute(text(query_bien_consumo))
data = result.fetchall()
conn_origen.close()
df = pd.DataFrame(data)
df

,movimiento_id,movimiento_uuid,movimiento_ref_uuid,fecha,documento_cod_serie,documento_cod_numero,concepto,entrada_cantidad,entrada_costo_uni,entrada_costo_tot,salida_cantidad,salida_costo_uni,salida_costo_tot
0,1,25082f63-2e5f-40fa-b63d-28959050c5eb,None,2025-05-18 18:32:37,MOV2025,1,inventario inicial,3.00,45.50,136.5000,None,NaN,NaN
1,2,16366708-522d-413e-b715-aa60c2ed87ee,None,2025-05-18 18:32:37,MOV2025,1,inventario inicial,3.00,45.50,136.5000,None,NaN,NaN
2,1,df9b451f-68b3-40ad-b136-ac7f36029403,None,2025-05-18 19:25:58,MOV2025,2,venta,None,None,None,1.00,0.0,0.0
3,2,9ec6697e-fc83-4a08-9a02-dc21dbaf2a20,None,2025-05-18 19:25:58,MOV2025,2,venta,None,None,None,2.00,0.0,0.0


In [ ]:
# CONTROLADORES KARDEX DETALLE
def crear_movimientos():
    data = [{
        # tipo KardexDetalle
    }]
    pass

def actualizar_movimientos():
    data = {
        'eliminar': [{
            # tipo KardexDetalle
        }],
        'crear': [{
            # tipo KardexDetalle
        }]
    }
    pass

def eliminar_movimientos():
    data = [{
        # tipo KardexDetalle
    }]
    pass

In [ ]:
# transaccion en las tablas utilizadas debe ser única y bloqueante de escritura ( no de lectura ), en tablas que no se utilizan no se bloquean
# transaccion de lectura siempre habilitado

# REPOSITORIOS KARDEX DETALLE
from typing import Any

from sqlmodel import Session, col, select
from app.entities.KardexMovimientoBienConsumoEntity import KardexMovimientoBienConsumoEntity
from app.services.proceso_kardex import get_movimiento_id
from app.services.proceso_movimientos import ProcesadorMovimientos


def crear_movimientos(session: Session, kardex_id: int, movimientos: list[dict[str,Any]]):
    
    # 1. Insertar movimientos
    movimiento_id = get_movimiento_id(session)
    movimiento_entities: list[KardexMovimientoBienConsumoEntity] = []
    for mov in movimientos:
        mov_entity = KardexMovimientoBienConsumoEntity(
            id=movimiento_id,
            kardex_bien_consumo_id=kardex_id,
            movimiento_uuid=mov["movimientosUuid"],
            movimiento_ref_uuid=mov["movimientoRefUuid"],
            movimiento_tipo=mov["movimientoTipo"],
            fecha=mov["fecha"],
            documento_fuente_cod_serie=mov["documentoFuenteCodigoSerie"],
            documento_fuente_cod_numero=mov["documentoFuenteCodigoNumero"],
            concepto=mov["concepto"],
            entrada_cant=mov["entradaCantidad"],
            entrada_costo_uni=mov["entradaCostoUnitario"],
            entrada_costo_tot=mov["entradaCostoTotal"],
            salida_cant=mov["salidaCantidad"],
            salida_costo_uni=mov["salidaCostoUnitario"],
            salida_costo_tot=mov["salidaCostoTotal"],
        )
        movimiento_entities.append(mov_entity)
        movimiento_id = movimiento_id + 1
    session.add_all(movimiento_entities)
    
    
    # 2. Obtener fecha minima
    fechas = [mov["fecha"] for mov in movimientos if mov.get("fecha")]
    if not fechas:
        return 
    fecha_minima = min(fechas)
    

    # 3. Procesar movimientos desde la fecha minima en lotes de 100
    offset = 0
    limit = 100
    
    while True:
        resultados = session.exec(
            select(KardexMovimientoBienConsumoEntity)
            .where(KardexMovimientoBienConsumoEntity.kardex_bien_consumo_id == kardex_id)
            .where(KardexMovimientoBienConsumoEntity.fecha >= fecha_minima)
            .order_by(col(KardexMovimientoBienConsumoEntity.fecha).asc(), col(KardexMovimientoBienConsumoEntity.id).asc())
            .offset(offset)
            .limit(limit)
        ).all()
        
        if not resultados:
            break
        
        df_movimientos = pd.DataFrame([r.model_dump() for r in resultados])

        if not df_movimientos.empty:
            procesador = ProcesadorMovimientos(df_movimientos, session)
            procesador.procesar()
            procesador.actualizar()
            offset += limit
        else:
            break


def actualizar_movimientos(movimientos_eliminar: list[dict[str,Any]], movimientos_crear: list[dict[str,Any]]):
    # eliminar_inventario()
    # iniciar_inventario()
    pass

def eliminar_movimientos(movimientos: list[dict[str,Any]]):
    pass

In [ ]:
from pandas import DataFrame


def procesar_entrada_bien_consumo_valor_nuevo( df: DataFrame, movimientos: list[object] ):
    pass

def procesar_entrada_bien_consumo_valor_salida():
    pass

def procesar_salida_bien_consumo_valor_nuevo():
    pass

def procesar_salida_bien_consumo_valor_entrada():
    pass

def procesar_nv_salida_bien_consumo():
    pass

def procesar_nv_servicio_reparacion_recurso_bien_consumo():
    pass